In [55]:
%reset -f
import os
from dotenv import load_dotenv
from datetime import datetime

# Get the SEO keywords and parent folder as input from the user
seo_keywords = "toubkal mountain "
parent_folder = "toubkal_mountain"
# 1: moroccoheritage 2 : gobitcode
website = "1"
current_date = datetime.now().strftime('%Y-%m-%d')

In [56]:
if website == '1':
    file_path = f'./format/moroccoheritage.mdx'
else:
    file_path = f'./format/gobitcode.mdx'

with open(file_path, 'r') as file:
    mdx_format = file.read()


mdx_format = mdx_format.replace("current_date", current_date)

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_cx = os.getenv('GOOGLE_CX')


In [57]:
from groq import Groq




client = Groq(api_key=groq_api_key)


completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": f"You are a blog writer who write blog in this format: \n {mdx_format} \n Do not include any introductory text "
        },
        {
            "role": "user",
            "content": f"Write a long-form blog about '{seo_keywords}' \n Do not forget image links and alt text"
        }
    ],
    temperature=1,
    max_tokens=1024*4,
    top_p=1,
    stream=None,
    stop=None,
)
# opencv cnn face recognition

mdx_blog = ""
for choice in completion.choices:
    mdx_blog += choice.message.content

print(mdx_blog)

---
title: Conquering the Mighty Toubkal Mountain: A Journey to the Roof of North Africa
date: '2024-11-28'  
tags: ['toubkal mountain', 'morocco trekking', 'Atlas Mountains']
draft: false
summary: Embark on an adventure of a lifetime as we explore the majestic Toubkal Mountain, the highest peak in North Africa, and discover the beauty and challenge of trekking in Morocco's Atlas Mountains.
---

## Introduction

Nestled in the heart of Morocco's Atlas Mountains, Toubkal Mountain stands tall and proud, beckoning adventurers and trekkers from around the world to conquer its majestic peak. As the highest point in North Africa, Toubkal Mountain offers a unique and unforgettable experience that combines breathtaking natural beauty, rich cultural heritage, and a challenging trek that will push your limits.

<TOCInline toc={props.toc} exclude="Introduction" />

## The Mighty Toubkal Mountain: A Brief Overview

Rising to an impressive 4,167 meters (13,671 feet) above sea level, Toubkal Mountai

In [58]:
import re
# Function to generate a slug from the title
def generate_slug(title):
    # Convert to lowercase
    slug = title.lower()
    # Replace spaces and special characters with hyphens
    slug = re.sub(r'[^a-z0-9]+', '-', slug)
    # Remove leading and trailing hyphens
    slug = slug.strip('-')
    return slug


# Regular expression to extract the title
title_match = re.search(r'^title:\s*(.*)$', mdx_blog, re.MULTILINE)

if title_match:
    title = title_match.group(1)
    slug = generate_slug(title)
else:
    print("Title not found")

print(f"Slug: {slug}")

Slug: conquering-the-mighty-toubkal-mountain-a-journey-to-the-roof-of-north-africa


In [59]:
from google_images_search import GoogleImagesSearch

def download_images(keyword,path,  limit=1):
    # You need to get your own API key and CX from Google Custom Search JSON API
    gis = GoogleImagesSearch(google_api_key,google_cx)

    search_params = {
        'q': keyword,
        'num': limit,
        'safe': 'off',
        'fileType': 'jpg|png',
        'imgType': 'photo',
        'imgSize': 'LARGE',  # Use a single size or remove this line if not needed
        # 'imgDominantColor': 'black'  # Use a single color or remove this line if not needed
    }

    gis.search(search_params=search_params)
    image = gis.results()[0]
    image.download(f'./output/{path}')
    image_name = os.path.basename(image.path)
    return image_name




In [60]:



# Regex to match the <img> tag and capture the alt attribute
img_tag_regex = r'<img[^>]*alt="([^"]*)"[^>]*>'

# Find all matches
matches = re.findall(img_tag_regex, mdx_blog)



# Print the alt attributes
for match in matches:
    image_name =  download_images(match , parent_folder)
    print(f"alt attribute: {match}")
    new_src = f'/static/images/{parent_folder}/{image_name}'
    mdx_blog = re.sub(rf'(<img[^>]*src=")[^"]*("[^>]*alt="{re.escape(match)}"[^>]*>)', rf'\1{new_src}\2', mdx_blog)
    
print(mdx_blog) 
    

alt attribute: Toubkal Mountain Overview
alt attribute: Toubkal Mountain Trek Preparation
alt attribute: Toubkal Mountain Trek Views
alt attribute: Toubkal Mountain Adventure
---
title: Conquering the Mighty Toubkal Mountain: A Journey to the Roof of North Africa
date: '2024-11-28'  
tags: ['toubkal mountain', 'morocco trekking', 'Atlas Mountains']
draft: false
summary: Embark on an adventure of a lifetime as we explore the majestic Toubkal Mountain, the highest peak in North Africa, and discover the beauty and challenge of trekking in Morocco's Atlas Mountains.
---

## Introduction

Nestled in the heart of Morocco's Atlas Mountains, Toubkal Mountain stands tall and proud, beckoning adventurers and trekkers from around the world to conquer its majestic peak. As the highest point in North Africa, Toubkal Mountain offers a unique and unforgettable experience that combines breathtaking natural beauty, rich cultural heritage, and a challenging trek that will push your limits.

<TOCInline t

In [61]:
#correct : in title
# Sample string


# Regex to match and remove the colon from the title line
title_regex = r'^(title:.*?):(.*)$'

# Remove the colon from the title line
mdx_blog = re.sub(title_regex, r'\1\2', mdx_blog, flags=re.MULTILINE)

print(mdx_blog)

---
title: Conquering the Mighty Toubkal Mountain A Journey to the Roof of North Africa
date: '2024-11-28'  
tags: ['toubkal mountain', 'morocco trekking', 'Atlas Mountains']
draft: false
summary: Embark on an adventure of a lifetime as we explore the majestic Toubkal Mountain, the highest peak in North Africa, and discover the beauty and challenge of trekking in Morocco's Atlas Mountains.
---

## Introduction

Nestled in the heart of Morocco's Atlas Mountains, Toubkal Mountain stands tall and proud, beckoning adventurers and trekkers from around the world to conquer its majestic peak. As the highest point in North Africa, Toubkal Mountain offers a unique and unforgettable experience that combines breathtaking natural beauty, rich cultural heritage, and a challenging trek that will push your limits.

<TOCInline toc={props.toc} exclude="Introduction" />

## The Mighty Toubkal Mountain: A Brief Overview

Rising to an impressive 4,167 meters (13,671 feet) above sea level, Toubkal Mountain

In [62]:
filename = f"./output/{slug}.mdx"
with open(filename, 'w') as file:
    file.write(mdx_blog)

print(f"File '{filename}' created successfully.")

File './output/conquering-the-mighty-toubkal-mountain-a-journey-to-the-roof-of-north-africa.mdx' created successfully.
